In [8]:
import importlib
import time
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
# import seaborn as sns
import scipy.stats
import math
import numpy as np
import colorsys

import src.analysis_utils as au
import src.load_functions as lf
import src.distance_matrix_experiment as dme

import rpy2
import rpy2.robjects
import rpy2.robjects.packages

# If TreeDist is not installed, see the previous step 4
treedist = rpy2.robjects.packages.importr('TreeDist')
ape = rpy2.robjects.packages.importr('ape')

res = lf.ResourcesManager()
ethnologue_tree = res.ethnologue_tree
languages_all = res.indoeuropean_languages_with_data
wiki_articles_df = res.wiki_articles_df

def filename_permutations_to_metadata(filename):
    split = filename.split('.')
    metadata = {
        'number_of_languages' : int(split[2].replace('langs', '')),
        'embedding_metric': split[3],
        'persistent_diagram_metric': split[4],
        'persistent_diagram_dim': int(split[5].replace('d', '')),
        'tree_algorithm': split[6],
        'number_of_permutations': int(split[7].replace('len', ''))
    }
    
    return metadata

def lighten_color(color, lscale):
    color_hls = colorsys.rgb_to_hls(*color[:3])
    return colorsys.hls_to_rgb(color_hls[0], lscale * color_hls[1], color_hls[2])

# Load the permutation test data

In [ ]:
ts = time.perf_counter()
experiment_name = 'ethnologue_10k_d2'
table_source = 'tree leaf permutations'
tree_distances_folder = Path(f'data/tree-distances/{experiment_name}/')
data_folder = Path('data/random_permutations_data_frames/ethnologue_10k_d2/')
number_of_permuations = 10
experiment_name = 'ethnologue_10k_d2'

#=========================================================================

parameters_list = []
for tree_algorithm in ['nj', 'upgma']:
    for embedding_metric in ['euclidean', 'cosine']:
        for persistent_diagram_dim in [0,1,2]:
            for persistent_diagram_metric in ['bars_statistics', 'bottleneck', 'persistence_image', 'sliced_wasserstein']:
                name = (f'{tree_algorithm} ' +
                        f'{embedding_metric[:3]} {persistent_diagram_dim} ' +
                        f'{"-".join([w[:3] for w in persistent_diagram_metric.split("_")])}')
                filename = f'permutations.{experiment_name}.langs<NUMBER_OF_LANGUAGES>.{embedding_metric}.{persistent_diagram_metric}.d{persistent_diagram_dim}.{tree_algorithm}.len{number_of_permuations}.csv'
                parameters_list.append({
                    'index' : name,
                    'filename' : filename,
                })

tree_distances_df = pd.read_csv(tree_distances_folder / 'tree_distances_df.csv')
table_std_dfs = {}  # stds away from mean
table_gta_dfs = {}  # greater than algorithm
for num in (30, 50, 81):
    table_std = []
    table_gta = []
    for parameters in parameters_list:
        filename = parameters['filename'].replace('<NUMBER_OF_LANGUAGES>', str(num))
        df = pd.read_csv(data_folder / filename)
        metadata = filename_permutations_to_metadata(filename)
        
        real_distances = tree_distances_df[
            (tree_distances_df.tree_algorithm == metadata['tree_algorithm']) &
            (tree_distances_df.embedding_metric == metadata['embedding_metric']) &
            (tree_distances_df.persistent_diagram_dim == metadata['persistent_diagram_dim']) &
            (tree_distances_df.persistent_diagram_metric == metadata['persistent_diagram_metric']) &
            (tree_distances_df.number_of_languages == num)
        ]
        
        table_std_row = {'index' : parameters['index']}
        table_gta_row = {'index' : parameters['index']}
        for distance in ['dist_jrf_k1', 'dist_jrf_k2', 'dist_phylo', 'dist_clust', 'dist_match', 'dist_path']:
            if len(real_distances[distance]) != 1:
                raise ValueError("Extracting distance from table failed: did not get exactly 1 row.")
            actual_distance = real_distances[distance].iloc[0]
            
            mean, std = np.mean(df[distance]), np.std(df[distance])
            std_from_mean = (mean - actual_distance) / std
            table_std_row[distance] = std_from_mean
            
            table_gta_row[distance] = sum(df[distance] > actual_distance)
            
        table_std.append(table_std_row)
        table_gta.append(table_gta_row)
    table_std_dfs[num] = pd.DataFrame(table_std).set_index('index')
    table_gta_dfs[num] = pd.DataFrame(table_gta).set_index('index')

print(f'Time: {time.perf_counter() - ts : .2f} s')

# Plot the results

In [ ]:
fig, axs = plt.subplot_mosaic('abc;ddd', figsize=(18, 15), height_ratios=(14,1), sharey=False)
data = table_std_dfs
data_min = min(tab.min().min() for tab in data.values())
data_max = max(tab.max().max() for tab in data.values())
distances = [f'dist_{dist}' for dist in ['jrf_k1', 'jrf_k2', 'match', 'phylo', 'clust', 'path']]

colors = {'bot': lighten_color(mpl.colors.to_rgb(mpl.colors.TABLEAU_COLORS['tab:purple']), .85),
          'sli-was':lighten_color(mpl.colors.to_rgb(mpl.colors.TABLEAU_COLORS['tab:green']), 1),
          'per-ima':lighten_color(mpl.colors.to_rgb(mpl.colors.TABLEAU_COLORS['tab:blue']), 1.5),
          'bar-sta':lighten_color(mpl.colors.to_rgb(mpl.colors.TABLEAU_COLORS['tab:orange']), 1.3)}
colors_dark = {pdm : tuple(x*.8 for x in color[:3]) for pdm, color in colors.items()}
fillstyles = {'nj': 'top', 'upgma': 'none'}
marker_alpha = .8

for num, ax in zip((30, 50, 81), [axs['a'], axs['b'], axs['c']]):
    height = 0
    heights = []
    labels = []
    values = {
        'bar-sta' : {'nj' : [], 'upgma' : []},
        'bot' : {'nj' : [], 'upgma' : []},
        'per-ima' : {'nj' : [], 'upgma' : []},
        'sli-was' : {'nj' : [], 'upgma' : []}
    }
    values_alternative = {
        'bar-sta' : {'nj' : [], 'upgma' : []},
        'bot' : {'nj' : [], 'upgma' : []},
        'per-ima' : {'nj' : [], 'upgma' : []},
        'sli-was' : {'nj' : [], 'upgma' : []}
    }
    splitter_heights = []
    splitter_labels = []
    
    
    for embedding_distance in ['euc', 'cos']:
        for persistence_diagram_dimension in [0, 1, 2]:
            height+= .5
            splitter_heights.append(height)
            splitter_labels.append(f'{"euclidean" if embedding_distance=="euc" else "cosine"}, dim {persistence_diagram_dimension}')
            height+= 1
            for distance in distances: #sorted(table_std_dfs[num].columns):
                labels.append(distance)
                heights.append(height)
                height+= 1
                for persistent_diagram_metric in ['bar-sta', 'bot', 'per-ima', 'sli-was']:
                    for tree_algorithm in ['nj', 'upgma']:
                        values[persistent_diagram_metric][tree_algorithm].append(
                            data[num][distance][f'{tree_algorithm} {embedding_distance} {persistence_diagram_dimension} {persistent_diagram_metric}'])
                        values_alternative[persistent_diagram_metric][tree_algorithm].append(
                            table_gta_dfs[num][distance][f'{tree_algorithm} {embedding_distance} {persistence_diagram_dimension} {persistent_diagram_metric}'])
                        
    # PLOTTING
    plt.sca(ax)
    for height in splitter_heights:  # splitter lines
        plt.axhline(height, color='black', linewidth=.5)
    
    plt.grid(alpha=.2)
    plt.axvline(0, color='black', linestyle='-', linewidth=1, alpha=.4, label='mean')  # std 0 line
    plt.axvline(2, color='black', linestyle='--', linewidth=1, alpha=.4, label='std=2')  # std 0 line
    plt.axvline(4, color='black', linestyle='dotted', linewidth=1, alpha=.4, label='std=4')  # std 0 line
    
    for tree_algorithm in ['nj', 'upgma']:
        for persistent_diagram_metric in ['bar-sta', 'bot', 'per-ima', 'sli-was']:
            plt.plot(values[persistent_diagram_metric][tree_algorithm], heights,
                     linestyle='none', marker='o', color=colors[persistent_diagram_metric],
                     fillstyle=fillstyles[tree_algorithm], alpha=marker_alpha, markersize=9, markeredgewidth=2,
                     label=f'{tree_algorithm} {persistent_diagram_metric}')
            # HIGHLIGHT dosts with large gta
            for val, val_alt, height in zip(
                values[persistent_diagram_metric][tree_algorithm],
                values_alternative[persistent_diagram_metric][tree_algorithm],
                heights
            ):
                better_than = 99500
                if val_alt >= better_than:
                    plt.plot([val], [height],
                             linestyle='none', marker='*', color=colors_dark[persistent_diagram_metric], alpha=.8, markersize=5,
                             fillstyle='full')
    
#     ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(base=1.0))
    plt.xticks(ticks=range(-2, 8), fontsize=12)
    
    plt.ylim(0, max(heights)+1)
    plt.xlim(-2, 7.5)
#     plt.xlim(data_min - (data_max-data_min)/25, data_max + (data_max-data_min)/25)
    ax.invert_yaxis()
    
    plt.xlabel('# standard deviations from mean', fontsize=14)
    plt.title(f'{num} languages', fontsize=16)
    
axs['a'].set_yticks(ticks=heights+splitter_heights, labels=labels+splitter_labels, fontsize=14)
axs['b'].set_yticks(ticks=heights+splitter_heights, labels=[None]*len(heights+splitter_heights))
axs['c'].set_yticks(ticks=heights+splitter_heights, labels=[None]*len(heights+splitter_heights))
    
#LEGEND
plt.sca(axs['d'])
plt.xticks([])
plt.yticks([])
plt.box(False)
plt.legend(
    [
        mpl.lines.Line2D([0], [0], linestyle='none', marker='o', markersize=9, markeredgewidth=2,
                         fillstyle=fillstyles['upgma'], color=colors['bot']),
        mpl.lines.Line2D([0], [0], linestyle='none', marker='o', markersize=9, markeredgewidth=2,
                         fillstyle=fillstyles['nj'], color=colors['bot']),
        mpl.lines.Line2D([0], [0], linestyle='-', color='black', alpha=.4),
       
        mpl.lines.Line2D([0], [0], linestyle='none', marker='o', markersize=9, markeredgewidth=2,
                         fillstyle=fillstyles['upgma'], color=colors['sli-was']),
        mpl.lines.Line2D([0], [0], linestyle='none', marker='o', markersize=9, markeredgewidth=2,
                         fillstyle=fillstyles['nj'], color=colors['sli-was']),
        mpl.lines.Line2D([0], [0], linestyle='--', color='black', alpha=.4),
        
        mpl.lines.Line2D([0], [0], linestyle='none', marker='o', markersize=9, markeredgewidth=2,
                         fillstyle=fillstyles['upgma'], color=colors['per-ima']),
        mpl.lines.Line2D([0], [0], linestyle='none', marker='o', markersize=9, markeredgewidth=2,
                         fillstyle=fillstyles['nj'], color=colors['per-ima']),
        mpl.lines.Line2D([0], [0], linestyle='dotted', color='black', alpha=.4),
        
        mpl.lines.Line2D([0], [0], linestyle='none', marker='o', markersize=9, markeredgewidth=2,
                         fillstyle=fillstyles['upgma'], color=colors['bar-sta']),
        mpl.lines.Line2D([0], [0], linestyle='none', marker='o', markersize=9, markeredgewidth=2,
                         fillstyle=fillstyles['nj'], color=colors['bar-sta']),
        tuple(mpl.lines.Line2D([0], [0], color=colors_dark[c], linestyle='none',
                         marker='*', alpha=.8, markersize=5, fillstyle='full')
         for c in ['bot', 'sli-was', 'bar-sta', 'per-ima'])
    ],
    [
        'bottleneck, upgma',
        'bottleneck, nj',
        'mean',
        
        'sliced wasserstein, upgma',
        'sliced wasserstein, nj',
        'standard deviation = 2',
        
        'persistence image, upgma',
        'persistence image, nj',
        'standard deviation = 4',
        
        'bars statistics, upgma',
        'bars statistics, nj',
        r'better than $\geq$ ' + f'{better_than}'
    ],
    loc = (-.04, 0), ncol = 4, handler_map={tuple: mpl.legend_handler.HandlerTuple(ndivide=None)},
    handlelength=4,
    fontsize=14, frameon=False
)
    
plt.savefig(f"plots/std_comparison_plot_{table_source.replace(' ', '_')}_{number_of_permuations}.pdf", bbox_inches='tight')
plt.show()